In [34]:
import var_selection
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
pd.set_option('display.float_format', lambda x: '%.3f' % x)
np.set_printoptions(precision=4)

In [35]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

dataset = pd.read_csv('./data/hald_data.csv', delimiter='\t', skiprows=1, names=['Heat', 'Tricalcium_aluminate','Tricalcium_silicate','Tetracalcium_alumino_ferrite','Dicalcium_silicate'])
X = dataset[['Tricalcium_aluminate','Tricalcium_silicate','Tetracalcium_alumino_ferrite','Dicalcium_silicate']]
y = dataset[['Heat']]
corrMat = X.corr()
print('Covariate sample correlation matrix:\n'+str(corrMat))

Covariate sample correlation matrix:
                              Tricalcium_aluminate  Tricalcium_silicate  \
Tricalcium_aluminate                         1.000                0.229   
Tricalcium_silicate                          0.229                1.000   
Tetracalcium_alumino_ferrite                -0.824               -0.139   
Dicalcium_silicate                          -0.245               -0.645   

                              Tetracalcium_alumino_ferrite  Dicalcium_silicate  
Tricalcium_aluminate                                -0.824              -0.245  
Tricalcium_silicate                                 -0.139              -0.645  
Tetracalcium_alumino_ferrite                         1.000               0.297  
Dicalcium_silicate                                   0.297               1.000  


In [36]:
[Evals, Evecs] = np.linalg.eig(corrMat)
print('Eigenvalues of Covariate sample correlation matrix: \n'+str(Evals))

Eigenvalues of Covariate sample correlation matrix: 
[ 2.2009  1.2745  0.1508  0.3737]


In [37]:
if type(X) == pd.DataFrame:
    X = np.concatenate([X.as_matrix(), np.ones([X.shape[0],1])], axis = 1)
    y = y.as_matrix()

In [38]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

OLS_β = np.dot( np.linalg.inv(np.dot(X.T, X)), np.dot(X.T, y))
OLS_β_std = np.linalg.inv(np.dot(X.T, X))

In [39]:
delta_Heat = 20.
delta_Tricalcium_aluminate = 10.
delta_Tricalcium_silicate = 22.5
delta_Tetracalcium_alumino_ferrite = 8.5
delta_Dicalcium_silicate = 27.

In [40]:
τ_star = np.atleast_2d(delta_Heat / np.array([delta_Tricalcium_aluminate,delta_Tricalcium_silicate,delta_Tetracalcium_alumino_ferrite,delta_Dicalcium_silicate])).T

stats = np.concatenate([OLS_β[:-1], np.sqrt(np.atleast_2d(np.diag(OLS_β_std)[:-1]).T), τ_star], axis = 1)

stats = pd.DataFrame(stats, columns = ['OLS coefficient','OLS standard error', 'prior std τ*'], index = ['Tricalcium_aluminate','Tricalcium_silicate','Tetracalcium_alumino_ferrite','Dicalcium_silicate'])
print(stats)

                              OLS coefficient  OLS standard error  \
Tricalcium_aluminate                  1.69345             0.08980   
Tricalcium_silicate                   0.65953             0.02523   
Tetracalcium_alumino_ferrite          0.24612             0.08357   
Dicalcium_silicate                    0.00414             0.02604   

                              prior std τ*  
Tricalcium_aluminate               2.00000  
Tricalcium_silicate                0.88889  
Tetracalcium_alumino_ferrite       2.35294  
Dicalcium_silicate                 0.74074  


In [41]:
# Formulas found here: https://pdfs.semanticscholar.org/3ace/886849dd48eb911b0491d70ef3ec197f9234.pdf
SS_reg = (OLS_β.T.dot(X.T.dot(y)) - (1./X.shape[0])*(np.ones_like(y).T.dot(y))**2)[0,0]
SS_total = (y.T.dot(y) - (1./X.shape[0])*(np.ones_like(y).T.dot(y))**2)[0,0]
print('multiple correlation coef: {:.3f}'.format(SS_reg/SS_total))

multiple correlation coef: 0.982


In [45]:
β = np.array([0,0,0,0,0], dtype = float) #np.zeros([5,1])
ϵ_squared = 2.5**2
p = np.array([0.5,0.5,0.5,0.5,0]) #np.concatenate([0.5 * np.ones([4,1]), np.zeros([1,1])])
τ = np.concatenate([τ_star, 9 * np.ones([1,1])]) #np.ones_like(β)
λ = -1 * np.ones([5,1]) * np.infty
υ = np.ones([5,1]) * np.infty
ν = 0.01
iterations = 1000000

In [ ]:
# TN (lambda, υ)
results = var_selection.variable_selection(X, y, β, ϵ_squared, p, τ, ν, υ, λ, iterations)
pickle.dump(file = open("./results/hald/1000000/p_0.5_tau_1.pickle", 'wb'), obj=results)

/home/soren/Studium/MonteCarlo/mcmc_proj7/var_selection.py:36: RuntimeWarning: overflow encountered in exp
  bf = np.exp(β_bar**2/(2*var_star) - β_old**2/(2*τ**2))
